In [5]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

In [31]:
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
data_directory = './data/lunar/training/data/S12_GradeA/'

TRAINING_RELATION = 0.8

cat = pd.read_csv(cat_file)

76

In [89]:
cat

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


In [63]:
# Power, bandwidth, spectral entropy, peak frequency, dominant frequency, centroid frequency, mean frequency and spectral amplitude

def get_power(data):
    return np.sum(np.abs(data)**2)

def get_bandwidth(data, times):
    freqs = np.fft.fftfreq(len(times), times[1] - times[0])
    fft = np.fft.fft(data)
    return np.sum(np.abs(fft)**2)

def get_spectral_entropy(data):
    fft = np.fft.fft(data)
    power = np.abs(fft)**2
    power = power / np.sum(power)  # Normalización
    power = power[power > 0]  # Evitar log(0)
    return -np.sum(power * np.log(power))

def get_peak_frequency(data, times):
    freqs = np.fft.fftfreq(len(times), times[1] - times[0])
    fft = np.fft.fft(data)
    return np.abs(freqs[np.argmax(np.abs(fft))])

def get_dominant_frequency(data, times):
    freqs = np.fft.fftfreq(len(times), times[1] - times[0])
    fft = np.fft.fft(data)
    return np.abs(freqs[np.argmax(np.abs(fft))])

def get_centroid_frequency(data, times):
    freqs = np.fft.fftfreq(len(times), times[1] - times[0])
    fft = np.fft.fft(data)
    return np.sum(np.abs(fft) * np.abs(freqs)) / np.sum(np.abs(fft))

def get_mean_frequency(data, times):
    freqs = np.fft.fftfreq(len(times), times[1] - times[0])
    fft = np.fft.fft(data)
    return np.sum(np.abs(fft) * np.abs(freqs)) / np.sum(np.abs(fft))

def get_spectral_amplitude(data):
    fft = np.fft.fft(data)
    return np.max(np.abs(fft))

def get_characteristics(data, times):
    power = get_power(data)
    bandwidth = get_bandwidth(data, times)
    spectral_entropy = get_spectral_entropy(data)
    peak_frequency = get_peak_frequency(data, times)
    dominant_frequency = get_dominant_frequency(data, times)
    centroid_frequency = get_centroid_frequency(data, times)
    mean_frequency = get_mean_frequency(data, times)
    spectral_amplitude = get_spectral_amplitude(data)

    return dict(
        power=power,
        bandwidth=bandwidth,
        spectral_entropy=spectral_entropy,
        peak_frequency=peak_frequency,
        dominant_frequency=dominant_frequency,
        centroid_frequency=centroid_frequency,
        mean_frequency=mean_frequency,
        spectral_amplitude=spectral_amplitude
    )

def get_characteristics_from_file(filename):
    file = pd.read_csv(filename)
    times = np.array(file['time_rel(sec)'].tolist())
    data = np.array(file['velocity(m/s)'].tolist())

    return get_characteristics(data, times)

def get_mq_type(mq_type):
    if mq_type == 'impact_mq':
        return 0 # Impact
    elif mq_type == 'deep_mq':
        return 1 # Deep moonquake
    else:
        return 2 # Shallow moonquake

def get_dataset_characteristics():
    characteristics = []
    limit = cat.shape[0]

    for i in range(1, limit):
        row = cat.iloc[i]
        filename = row.filename
        mq_type = row.mq_type

        csv_file = f'{data_directory}{filename}.csv'
        data_cat = pd.read_csv(csv_file)

        csv_times = np.array(data_cat['time_rel(sec)'].tolist())
        csv_data = np.array(data_cat['velocity(m/s)'].tolist())

        characteristics.append(dict(
            mq_type=get_mq_type(mq_type),
            **get_characteristics(csv_data, csv_times)
        ))
        print(f'Processed training file {i}/{limit - 1} ({filename})')

    return pd.DataFrame(characteristics)

In [64]:
dataset_characteristics = get_dataset_characteristics()

Processed training file 1/75 (xa.s12.00.mhz.1970-03-25HR00_evid00003)
Processed training file 2/75 (xa.s12.00.mhz.1970-03-26HR00_evid00004)
Processed training file 3/75 (xa.s12.00.mhz.1970-04-25HR00_evid00006)
Processed training file 4/75 (xa.s12.00.mhz.1970-04-26HR00_evid00007)
Processed training file 5/75 (xa.s12.00.mhz.1970-06-15HR00_evid00008)
Processed training file 6/75 (xa.s12.00.mhz.1970-06-26HR00_evid00009)
Processed training file 7/75 (xa.s12.00.mhz.1970-07-20HR00_evid00010)
Processed training file 8/75 (xa.s12.00.mhz.1970-07-20HR00_evid00011)
Processed training file 9/75 (xa.s12.00.mhz.1970-09-26HR00_evid00013)
Processed training file 10/75 (xa.s12.00.mhz.1970-10-24HR00_evid00014)
Processed training file 11/75 (xa.s12.00.mhz.1970-11-12HR00_evid00015)
Processed training file 12/75 (xa.s12.00.mhz.1970-12-11HR00_evid00017)
Processed training file 13/75 (xa.s12.00.mhz.1970-12-27HR00_evid00019)
Processed training file 14/75 (xa.s12.00.mhz.1970-12-31HR00_evid00021)
Processed train

In [71]:
# model = Sequential([
#     Input(shape=(8,)),
#     Dense(16, activation='relu'),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

model = MLPClassifier(hidden_layer_sizes=(16, 16), activation='relu', solver='adam', max_iter=1000)


In [88]:
x = dataset_characteristics[['power', 'bandwidth', 'spectral_entropy', 'peak_frequency', 'dominant_frequency', 'centroid_frequency', 'mean_frequency', 'spectral_amplitude']]
y = dataset_characteristics[['mq_type']]
x

,power,bandwidth,spectral_entropy,peak_frequency,dominant_frequency,centroid_frequency,mean_frequency,spectral_amplitude
0,8.551625e-14,4.895044e-08,11.414261,0.828121,0.828121,0.558735,0.558735,0.000006
1,5.933450e-14,3.396372e-08,11.490417,0.828121,0.828121,0.657608,0.657608,0.000005
2,6.554142e-14,3.751689e-08,11.301749,0.828126,0.828126,0.715074,0.715074,0.000006
3,5.185712e-14,2.968359e-08,11.486610,0.828121,0.828121,0.665736,0.665736,0.000003
4,1.768520e-13,1.012333e-07,11.359430,0.828122,0.828122,0.835524,0.835524,0.000007
...,...,...,...,...,...,...,...,...
70,4.095598e-14,2.344394e-08,11.222442,0.828122,0.828122,0.815404,0.815404,0.000011
71,2.915963e-12,1.669164e-06,11.636065,0.828126,0.828126,0.992966,0.992966,0.000017
72,1.281508e-11,7.335581e-06,11.514368,0.394710,0.394710,0.846441,0.846441,0.000030
73,4.722268e-14,2.703078e-08,11.497791,0.828121,0.828121,0.665626,0.665626,0.000006


In [68]:
# history = model.fit(
#     training_characteristics[['power', 'bandwidth', 'spectral_entropy', 'peak_frequency', 'dominant_frequency', 'centroid_frequency', 'mean_frequency', 'spectral_amplitude']],
#     training_characteristics[['mq_type']],
#     epochs=100,
#     batch_size=4
# )

In [69]:
# test_loss, test_accuracy = model.evaluate(
#     test_characteristics[['power', 'bandwidth', 'spectral_entropy', 'peak_frequency', 'dominant_frequency', 'centroid_frequency', 'mean_frequency', 'spectral_amplitude']],
#     test_characteristics[['mq_type']]
# )

# print(f'Test accuracy: {test_accuracy * 100}%')
# print(f'Test loss: {test_loss}')


In [84]:
# 'min_samples_split': range(2, 18, 3) | 17, 5, 9, 2 MEJORES
# 'max_depth': [1, 3, 7, 12, 15, 18, 20, None] | 20
# 'max_features': [3, 4, 5, 6, None] | 5 MEJOR
# 'min_samples_leaf': range(3,18,3) | 6 MEJOR
# 'ccp_alpha':[X0.0, 0.001, 0.005,0.010, 0.015, 0.020, 0.025, 0.030, 0.035, 0.05] | 0.0 MEJOR

mlp_hyp_grid = {'hidden_layer_sizes': [(16, 8), (32, 32), (64, 64), (128, 128), (256, 256)],
                'activation': ['relu', 'tanh', 'logistic'],
                'solver': ['adam'],
            }
mdl_tree = GridSearchCV(MLPClassifier(), param_grid=mlp_hyp_grid, cv=10, scoring='balanced_accuracy', n_jobs=-1).fit(dataset_characteristics[['power', 'bandwidth', 'spectral_entropy', 'peak_frequency', 'dominant_frequency', 'centroid_frequency', 'mean_frequency', 'spectral_amplitude']], dataset_characteristics[['mq_type']])
mdl_tree.best_params_
mdl_tree.best_score_

c:\Users\maria\OneDrive\Escritorio\_\NSAC\space_apps_2024_seismic_detection\space_apps_2024_seismic_detection\.venv\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(
c:\Users\maria\OneDrive\Escritorio\_\NSAC\space_apps_2024_seismic_detection\space_apps_2024_seismic_detection\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.4666666666666666

In [83]:
mdl_tree.best_score_

0.4666666666666666